# Computing terrain parameters from Digital Elevation Model

Topographic parameters such as slope and aspect can be computed from a Digital Elevation Model (DEM) which is a representation of elevation data of the surface of the earth. In SOMOSPIE these terrrain parameters are used to downscale satellite-derived soil moisture data.

This notebook uses DEMs from [USGS 3DEP products](https://www.usgs.gov/3d-elevation-program/about-3dep-products-services) to compute 3 topographic parameters: Hillshading, Aspect and Slope.


The data from USGS standard DEMs is distributed in tiles that cover a fixed geographic area and these tiles can be merged to build a DEM for a larger region. 

<p align="center">
<img src="../../../somospie_pngs/tiles_to_mosaic.png" width="500"/>
</p>

<p align="center">
<b>Figure 1: </b>Merge multiple tiles into a mosaic.
</p>

Before running the workflow on this notebook, go to [USGS Data Download Application](https://apps.nationalmap.gov/downloader/#/elevation) and use the map to look for available DEM data. Once you have selected a specific region and resolution, you can get a txt file with all the individual download links for the tiles corresponding to your selection. This txt file will serve as input to this notebook which uses the links to download the tiles and merges them into a single DEM file to then compute terrain parameters using GDAL.

The terrain parameters are by default generated as GeoTIFF files, so this notebook includes the option to change their format and stack them if needed.

## Environment setup
Run the following code boxes to set the working directories and packages necessary for this workflow.

In [ ]:
from tools import *

In the code cell bellow specify the inputs to the workflow:
* **in_file:** path to the txt file with download links for DEM tiles you wish to use.
* **out_folder:** path to the folder you want the terrain parameters to be stored.
* **projection:** The projection can be an identifier such as 'EPSG:3572' or the path to a wkt file. To compute terrain parameters correctly, the DEM must be in a projection whose x, y and z coordinates are expressed in the same units, Albers Equal Area USGS projection was used for CONUS, but you can modify it depending on the region you are analyzing.

In [ ]:
in_file = './data.txt'
out_folder = '/media/volume/sdb/'
projection = 'albers_conus_reference.wkt'

In [ ]:
tiles_folder = os.path.join(out_folder, 'tiles')
Path(out_folder).mkdir(parents=True, exist_ok=True)
Path(tiles_folder).mkdir(parents=True, exist_ok=True)

## 1. Download DEM tiles
Downloads each tile from the URLs listed in input txt file and store them in the specified output folder on a subdirectory named tiles.

In [ ]:
print('Downloading tiles...')
download_dem(in_file, tiles_folder)
print('Download completed.')

## 2. Merge all DEM tiles in a single file
Merges downloaded tiles into a single raster (mosaic).

In [ ]:
raster_list = glob.glob(tiles_folder + '/*')
mosaic_path = os.path.join(out_folder, 'mosaic.tif')

merge_tiles(raster_list, mosaic_path)

# Optional: delete all tiles after building mosaic
shutil.rmtree(tiles_folder)

## 4. Reproject mosaic
To compute terrain parameters correctly, the DEM must be in a projection whose x, y and z coordinates are expressed in the same units. The next code cell reprojects the mosaic to ensure coordinates and elevation values are in meters (Albers Equal Area USGS projection was used for CONUS, but you can modify it depending on the region you are analyzing).

The projection can be an identifier such as 'EPSG:3572' or the path to a wkt file.

In [ ]:
dem_path = os.path.join(out_folder, 'elevation.tif')
reproject(mosaic_path, dem_path, projection)

# Optional: delete mosaic with initial projection
os.remove(mosaic_path)

## 5. Compute terrain parameters

GDAL is used to compute 3 terrain parameters: Hillshading, Aspect and Slope, from the DEM.

In [ ]:
compute_params_concurrently(out_folder, parameters=['slope', 'aspect', 'hillshading'])

## 6. Change raster format and projection (optional)
Terrain parameters will be stored by default in GeoTIFF format, run the following cells if you wish to get them in a different format.

Change the format and extension to the one you wish the files to be converted to in the next code cell. Go to [GDAL raster drivers](https://gdal.org/drivers/raster/index.html) to check which formats are available.

In [ ]:
raster_format = 'GTiff'
extension = '.tif'
projection = 'EPSG:4326'

In [ ]:
param_files = sorted(glob.glob(os.path.join(out_folder, '*.tif')))

for f in param_files:
    change_raster_format(f,  f[0:-4]+extension, raster_format)
    os.remove(f) # Optional: delete parameters with initial format

In [ ]:
param_files = sorted(glob.glob(os.path.join(out_folder, '*' + extension)))

for f in param_files:
    reproject(f, f, projection)

## 7. Create a stack with terrain parameters (optional)
If you want to get a stack of the terrain parameters run the following code box. The stack file will be stored in the ouput folder you specified at the start of this notebook.

In [ ]:
param_list = sorted(glob.glob(os.path.join(out_folder, '*' + extension)))
stack_file = os.path.join(out_folder ,'stack.tif')

build_stack(param_list, stack_file)